Лабораторная работа 2.

Реализовать обучения моделей и предсказание с их помощью в задаче классификации. 
Для каждой модели необходимо реализовать две функции trainNAME и predictNAME, где NAME имя модели. Данные функции имеют следующии сигнатуры: model = trainNAME(X, Y, ...{other params}...), predictNAME(model, X), где {other params} – это параметры определяемые студентом, включающие все используемые в модели константы, model – обученная модель, возвращаемая первой функцией. 
Функция предсказания должна возвращать вероятность класса 1. Все модели реализуются самостоятельно студентами, без использования библиотеки sklearn и подобных. 

В рамках лабораторной работы требуется реализовать: 
- Дерево решений (разбиение происходит на основании энтропийного критерия, предсказания внутри листа дерева – константное, критерий остановки – величина листа и глубина дерева) 
- Случайный лес (внутренняя модель – дерево решений) 
- Градиентный бустинг (внутреняя модель – дерево решений, псевдо-остатки считаются из логистической функции потерь «logistic loss», длина шага – константа, предсказание должно использовать logit преобразование) 

Инструкции по работе с лабой: 
1. Написать функции trainRND и predictRND для построния модели, возвращающуюся случайной вероятность принадлежности к классу 1 для каждой точки тестовой выборки. 
2. Подобрать выборку данных для задачи классификации на два класса (каждого класса не менее 20%, выборка содержит не менее 500 наблюдений). Разбить выборку данных на обучающую и тестовую выборку в пропорции 80:20.
3. Обучить на обучающей выборке и предсказать на тестовой выборке зависимую переменную с помощью случайной модели. Построить ROC-кривую для этой модели. 
4. Написать функции trainNAME и predictNAME для каждой из моделей 
5. Добавить ROC-кривые для каждой из моделей на тестовой выборке.

In [ ]:
#Импорт:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, roc_curve, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import matplotlib.pyplot as plt

In [ ]:
#Загрузим:
df = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')

In [ ]:
#Подготовка показателей:
df['age'] = 2020 - df['yr_built']
df['is_renovated'] = (df['yr_renovated'] != 0).astype(int) #был сделан капитальный ремонт
df['pricy'] = (df['price'] > 500000).astype(int)

#x и y:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
            'floors', 'waterfront', 'view', 'condition', 'grade',
            'sqft_above', 'sqft_basement', 'age', 'is_renovated']
x = df[features]
y = df['pricy']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [ ]:
#Дерево решений:
tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)
y_predicted_tree = tree.predict(x_test)
#tree_mse = mean_absolute_error(y_predicted_tree, y_test)
tree_acc = accuracy_score(y_test, y_predicted_tree)
print(f'Дерево решений (точность): {tree_acc:.3f}')

In [ ]:
#Случайный лес:
forest = RandomForestClassifier(criterion = 'entropy')
forest.fit(x_train, y_train)
y_predicted_forest = forest.predict(x_test)
forest_acc = accuracy_score(y_test, y_predicted_forest)
print(f'Случайный лес (точность): {forest_acc:.3f}')

In [ ]:
#Градиентный бустинг:
gradient = GradientBoostingClassifier(max_depth=4, n_estimators=200)
gradient.fit(x_train, y_train)
y_predicted_gradient = gradient.predict(x_test)
gradient_acc = accuracy_score(y_test, y_predicted_gradient)
print(f'Градиентный бустинг (точность): {gradient_acc:.3f}')

In [ ]:
#ROC-кривые:
def draw_roc_curve(y_test, y_pred_prob, name):
    fp, tp, thresholds = roc_curve(y_test, y_pred_prob)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fp, tp)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC кривая ({name})')
    plt.show()

y_prob_tree = tree.predict_proba(x_test)[:,1]
y_prob_forest = forest.predict_proba(x_test)[:,1]
y_prob_gradient = gradient.predict_proba(x_test)[:,1]
    
draw_roc_curve(y_test, y_prob_tree, 'дерево решений')
draw_roc_curve(y_test, y_prob_forest, 'случайный лес')
draw_roc_curve(y_test, y_prob_gradient, 'градиентный бустинг')

In [ ]:
#Экспорт дерева в .png
from sklearn.tree import export_graphviz
export_graphviz(tree, out_file='tree.dot', feature_names = x.columns.tolist(), class_names=['Дорогой','Недорогой'],
           rounded = True, proportion = False, precision = 0, filled = True)
!dot -Tpng tree.dot -o tree.png 
from IPython.display import Image
Image(filename = 'tree.png')